In [1]:
import numpy as np
import ot
import torch
from math import sqrt

from aux_functions import *
from TheGAN import LevyGAN
import configs

In [2]:
# print(read_serial_number("model_G1_D1_gp_Hsym_4d_62noise"))

In [3]:
config = configs.config

training_config = configs.training_config

levG = LevyGAN()
levG.load_dicts_unstructured('model_saves/GAN2_2d_randomly_excellent_generator.pt', 'model_saves/GAN2_2d_62noise_randomly_excellent_max_discriminator.pt')
config['unfixed test batch size'] = 65536
levG.reload_testing_config(config)
levG.do_tests(comp_joint_err=False)
print(levG.make_report())

UNFIXED PART OF REPORT TIME: 0.16226464600185864
CHEN ERRORS TIME: 0.004960499001754215


/home/andy/anaconda3/envs/GAN/lib/python3.10/site-packages/ot/lp/solver_1d.py:41: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484746364/work/aten/src/ATen/native/TensorShape.cpp:2981.)
  cws = cws.T.contiguous()


discr grad norm:  1.12319, discr loss: -1.37461
errs: ['0.1139', '0.0980', '0.1173', '0.1081', '0.1115', '0.1048', '0.1131', '0.0808'], chen errs: 0.8694
